In [1]:
# 필요한 라이브러리 불러오기

from konlpy.tag import Mecab 

import pandas as pd
import numpy as np

from collections import Counter

import warnings
warnings.filterwarnings('ignore')

dir = 'C:/Users/juwan/Desktop/산학프로젝트/using_data/'

In [2]:
# mecab 실행

mecab = Mecab(dicpath=r"C:\Mecab\mecab-ko-dic") # dictation path 설정 / mecab-ko-dic이 mecab의 단어사전임

In [3]:
# 형태소 분석할 데이터 준비 - 정책 내용 데이터 불러오기 

tagging = pd.read_csv(dir + 'content_tagging.csv', encoding = 'utf-8')

In [4]:
# name, description, content의 모든 내용으로 단어 추출

nouns_list = []

for i in range(len(tagging['content'])):
    nouns_list.extend(mecab.nouns(tagging['content'][i]))

In [5]:
# 불용어 제거(1) - 2글자 이상인 단어만 추출

nouns_list = [i for i in nouns_list if len(i) >= 2]

In [6]:
# 행 전체 다 보기

pd.set_option('display.max_rows',None)

In [7]:
word_df_cnt = pd.read_csv(dir + 'word_df_cnt.csv', encoding = 'utf-8')

### 사용자 정의 함수 

#### 1. score함수 
- 각 카테고리 마다의 점수를 합하여 리스트로 반환

In [10]:
def score(text): # input: 문자열 텍스트
    
    nl = mecab.nouns(text)
    
    score_money = 0
    score_interview = 0
    score_edu = 0
    score_job = 0
    score_consult = 0

    for i in nl:
        if i in list(word_df_cnt['description_word']):
            score_money += int(word_df_cnt.loc[word_df_cnt['description_word'] == i, 'pnt_money'])
            score_interview += int(word_df_cnt.loc[word_df_cnt['description_word'] == i, 'pnt_interview'])
            score_edu += int(word_df_cnt.loc[word_df_cnt['description_word'] == i, 'pnt_edu'])
            score_job += int(word_df_cnt.loc[word_df_cnt['description_word'] == i, 'pnt_job'])
            score_consult += int(word_df_cnt.loc[word_df_cnt['description_word'] == i, 'pnt_consult'])
        else:
            continue
    return [score_money, score_interview, score_edu, score_job, score_consult]

#### 2. whatcategory함수 
- score함수로 만든 점수 리스트를 넣으면 어떤 카테고리가 가장 높은 점수인지 출력(복수개면 복수개 출력)

In [11]:
def whatcategory(score_list):
    category = []
    for i in range(len(score_list)):
        if score_list[i] == max(score_list):
            if i == 0:
                category.append('money')
            if i == 1:
                category.append('interview')
            if i == 2:
                category.append('edu')
            if i == 3:
                category.append('job')
            if i == 4:
                category.append('consult')
    return category

In [12]:
whatcategory(score('안녕하세요 저는 빅데이터 분야로 취업하고 싶어 교육을 신청하는 사람입니다'))

['edu']

### 테스트

In [13]:
nums = []
text_list = tagging.loc[(tagging['index'] == 34)|(tagging['index'] == 48)|(tagging['index'] == 30)|(tagging['index'] == 62)|(tagging['index'] == 72)|(tagging['index'] == 82)|(tagging['index'] == 87)|(tagging['index'] == 127)|(tagging['index'] == 137)|(tagging['index'] == 295),'content']
text_list= text_list.reset_index(drop=True)
text_list = list(text_list)

In [14]:
cat = []
for i in text_list:
    cat.append(whatcategory(score(i)))

In [16]:
text_df = tagging.loc[(tagging['index'] == 34)|(tagging['index'] == 48)|(tagging['index'] == 30)|(tagging['index'] == 62)|(tagging['index'] == 72)|(tagging['index'] == 82)|(tagging['index'] == 87)|(tagging['index'] == 127)|(tagging['index'] == 137)|(tagging['index'] == 295)]
text_df = text_df[['name','description','content']]
text_df['predict_cat'] = [', '.join(i) for i in cat]

In [17]:
text_df['actual_cat'] = ['money','interview','interview','money','job','job','edu','edu','consult','consult']

In [20]:
counseling = pd.read_csv(dir + 'counseling.csv', encoding = 'euc-kr')

In [22]:
pred_cat = []

for i in list(counseling['title'] + ' ' + counseling['text']):
    pred_cat.append(whatcategory(score(i)))

In [23]:
counseling['pred_cat'] = [', '.join(i) for i in pred_cat]

In [24]:
pd.options.display.max_colwidth = 2000

In [25]:
lisst = [i.split(', ') for i in counseling['tag']]

for i in lisst:
    for j in range(len(i)):
        if i[j] == '취업상담':
            i[j] = 'consult'
        if i[j] == '면접지원':
            i[j] = 'interview'
        if i[j] == '일자리':
            i[j] = 'job'
        if i[j] == '지원금':
            i[j] = 'money'
        if i[j] == '교육지원':
            i[j] = 'edu'

In [85]:
counseling['tag2'] = [', '.join(i) for i in lisst]

In [34]:
counseling.shape

(342, 4)